### MULTIPLE RUNS: Import Libraries, READ SWMM INP file, Setup MongoDB Database to store results

In [6]:
import yaml
from swmm_objects_3 import *
from swmm_read_3 import *
from swmm_run_3 import *
from pymongo import MongoClient
from random import randint

# Constants:
squareFeetInOneAcre = 43560.0
# Specify important files and parameters for the location:
swmmInitialInpFileNameStr = "wingohocking_PWD_y12m08d29-y12m09d05.inp"  # initial SWMM .inp file
# treatmentRatio = 0.081   # cfs/imperv_acre calibrated for Wingohocking T-14 outfall 7/1/12 - 6/30/13
lidDictFileNameStr = "LID_Dict.yaml"
# Calculate parameters for the watershed from the SWMM model INP file:
# watershedImpervAreaAcre = total_impervious_area(swmmInitialInpFileNameStr)
# Load YAML files 
f = open(lidDictFileNameStr,'r')   
lidDict = yaml.load(f)           # Read the LID ContribImpervArea and GreenedAcres
f.close()
# Compute a dictionary of contributing impervious ACRES for each LID 
contribImpervAreaAcre = {}
print("The following LID names are available:")
for lid in lidDict:
    print(lid)
    contribImpervAreaAcre[lid] = lidDict[lid]['ContribImpervArea'] / squareFeetInOneAcre  # acres
# Open the specified SWMM zeroLID INP file and obtain necessary data from it for multiple runs:    
swmmInitialInpStr = read_inp_file(swmmInitialInpFileNameStr)

# Specify MongoDB database server to use and login with credentials:    
mongoServer = 'mongodb://localhost:27017/'  # localhost
database = "fengwei"
client = MongoClient(mongoServer)
db = client[database]   # use or create the database
# db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database

collectionName = 'testrun2'
thisRunCollection = db[collectionName]    # use or create the collection

The following LID names are available:
RainBarrel
MorrisTT
wakefield_BR_RG


# Generate Zero LID Model Object & View the list of subcatchments where LID can be placed:

In [7]:
(section_names,sections) = read_inp(swmmInitialInpStr)
initialModel = swmm_model("InitialModel",section_names,sections)
lidShedsList = initialModel.getLidSubcatchmentNameList()
print("The following subcatchments can receive LIDs:")
print(lidShedsList)

The following subcatchments can receive LIDs:
['TT14-574_shed', 'T14-K3_shed', 'T14-B_shed', 'TT14-720_shed', 'TT14-646A_shed', 'TT14-552B_shed', 'T14-R6_shed', 'TT14-296B_shed', 'T14-E_shed', 'T14-J_shed', 'TT14-584_shed', 'T14-A2_shed', 'TT14-558_shed', 'TT14-502_shed', 'T14-P1_shed', 'T14-L2_shed', 'T14-A1_shed', 'T14-A3_shed', 'TT14-262A_shed', 'T14-P3_shed', 'TT14-620B_shed', 'T14-A4_shed', 'TT14-530_shed', 'TT14-524_shed', 'TT14-291D_shed', 'TT14-736_shed', 'TT14-708_shed', 'T14-D_shed', 'T14-M2_shed', 'T14-C1_shed', 'T14-R7_shed', 'T14-R1_shed', 'T14-C2_shed', 'T14-R5_shed', 'T14-M1_shed', 'T14-R2_shed', 'T14-P4_shed', 'TT14-706_shed', 'T14-G_shed', 'TT14-610C_shed', 'TT14-746_shed', 'TT14-740_shed', 'TT14-648A_shed', 'TT14-552A_shed', 'TT14-702_shed']


# Run a test simulation to check the software

In [3]:
# Run SWMM on the initial Zero LID model input:
swmmRptDict = run_swmm_and_return_results(swmmInitialInpStr)
#(section_names,sections) = read_inp(swmmInpStr)
#model = swmm_model("Model",section_names,sections)

0.0 min, 2.64 sec


# Print selected outputs from the test simulation

In [4]:
print("Sewer Outflow Time Series:\n",swmmRptDict["outflow_seriesCFS"])

Sewer Outflow Time Series:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [9]:
(section_names,sections) = read_inp(swmmInitialInpStr)
initialModel = swmm_model("InitialModel",section_names,sections)

lidShedsList = initialModel.getLidSubcatchmentNameList()


subcatAreaDict = {}            # Total area in each subcatchment (never changes)
subcatImpPctDict = {}          # ORIGINAL Impervious percentage in each subcatchment (changes when LID added)
subcatTreatableAreaDict = {}   # Original IMPERVIOUS area in zero LID SWMM file
#subcatMaxLID = {}              # Maximum number of each LID that can be installed in each subcatchment
subcatMaxGI = {}               # Maximum greened acres for each LID that can be installed in each subcatchment
# Now calculate the dictionaries defined above from data in the zero LID SWMM input file:

lidShedsList = ['T14-B_shed','TT14-296B_shed']  # specify a subset of the LID subcatchments if desired
for lidShed in lidShedsList:
    subcatArea = initialModel.getSubcatchmentArea(lidShed)
    subcatAreaDict[lidShed] = subcatArea
    pctImperv = initialModel.getSubcatchmentPctImperv(lidShed)
    subcatImpPctDict[lidShed] = pctImperv
    subcatTreatableAreaDict[lidShed] = subcatArea*pctImperv/100.0
#print(subcatTreatableAreaDict['TT14-574_shed'])    

maxTreatableFraction = 0.91
max_run = 500
lid = 'wakefield_BR_RG'
contribImpervArea = contribImpervAreaAcre[lid]  # acres

for lidShed in lidShedsList:
    print('In subshed ',lidShed,'using LID ',lid)
    #reinitialize the model with the original zero LID swmmInpStr:
    (section_names,sections) = read_inp(swmmInitialInpStr)  # Start with the initial NO LID SWMM input file
    model = swmm_model("Model",section_names,sections)   # This model will be modified below in the loops
    lidNumberBySubcatAndLidDict = {}

    subcatTreatableArea = maxTreatableFraction*subcatTreatableAreaDict[lidShed]   
    # set initial values before placing LIDs:
    subcatAreaTreated = 0.0
    numLID = 0
    numLIDIncrement = 1
    run = 0
    while run <= max_run:
        lidNumberByLidDict = {}
        subcatAreaTreated = subcatAreaTreated + numLID*contribImpervArea
        if subcatAreaTreated > subcatTreatableArea:
            break
        lidNumberByLidDict[lid] = numLID
        lidNumberBySubcatAndLidDict[lidShed] = lidNumberByLidDict
        model.lidChangeNumber(lidShed,lid,numLID)
        area = model.getLidArea(lidShed,lid)   # the lid footprint in the subcatchment 
        model.lidChangeArea(lidShed,lid,area,contribImpervAreaAcre[lid])  # must call to properly set FromImp parameter
        swmmInputFileStr=model.output()  # A modified SWMM input file containing the changed LID numbers
        totalGreenedAcreByLidDict = greened_acres_deployed(swmmInputFileStr,lidDict) # extract greened acres for each LID

        totalGreenedAcres = 0.0
        for l in totalGreenedAcreByLidDict:
            totalGreenedAcres += totalGreenedAcreByLidDict[l]
            
        # RUN SWMM and read it's output file creating a Report Dictionary
        swmmRptDict = run_swmm_and_return_results(swmmInputFileStr)
        
        # Append additional dictionaries to the Report Dictionary:
        swmmRptDict["lidDict"] = lidDict
        swmmRptDict["lidNumberBySubcatAndLidDict"] = lidNumberBySubcatAndLidDict
        
        # Insert the augmented report dictionary into the Mongo Database Collection specified above:
        docId = thisRunCollection.insert_one(swmmRptDict).inserted_id
        
        # Show user the progress through the runs:
        print("Run ",run," completed:")
        print("Stored record for a run with manual LID assignments with docId = ")
        print(docId)
        greenAcrePrintStr = 'Total Greened Acres = {:0.1f} with:'.format(totalGreenedAcres)
        lidGreenedAcreStr = ''
        for l in totalGreenedAcreByLidDict:
            lidGreenedAcreStr += " {:s} = {:0.1f},".format(l,totalGreenedAcreByLidDict[l])
        print(greenAcrePrintStr + lidGreenedAcreStr)
        numLID = numLID + numLIDIncrement
        run = run + 1
    print()


In subshed  T14-B_shed using LID  wakefield_BR_RG
0.0 min, 3.23 sec
Run  0  completed:
Stored record for a run with manual LID assignments with docId = 
5afcca24fa005d3274fa53d8
Total Greened Acres = 0.0 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.19 sec
Run  1  completed:
Stored record for a run with manual LID assignments with docId = 
5afcca27fa005d3274fa53d9
Total Greened Acres = 0.7 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 0.7,
0.0 min, 3.05 sec
Run  2  completed:
Stored record for a run with manual LID assignments with docId = 
5afcca2afa005d3274fa53da
Total Greened Acres = 1.5 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 1.5,
0.0 min, 3.30 sec
Run  3  completed:
Stored record for a run with manual LID assignments with docId = 
5afcca2efa005d3274fa53db
Total Greened Acres = 2.2 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 2.2,
0.0 min, 3.37 sec
Run  4  completed:
Stored record for a run with manual LID assig

0.0 min, 2.66 sec
Run  9  completed:
Stored record for a run with manual LID assignments with docId = 
5afcca9ffa005d3274fa53fe
Total Greened Acres = 6.7 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 6.7,
0.0 min, 3.08 sec
Run  10  completed:
Stored record for a run with manual LID assignments with docId = 
5afccaa2fa005d3274fa53ff
Total Greened Acres = 7.5 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 7.5,
0.0 min, 2.88 sec
Run  11  completed:
Stored record for a run with manual LID assignments with docId = 
5afccaa5fa005d3274fa5400
Total Greened Acres = 8.2 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 8.2,
0.0 min, 3.07 sec
Run  12  completed:
Stored record for a run with manual LID assignments with docId = 
5afccaa8fa005d3274fa5401
Total Greened Acres = 9.0 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 9.0,
0.0 min, 2.67 sec
Run  13  completed:
Stored record for a run with manual LID assignments with docId = 
5afccaabfa005d3274fa5402
